### GreedyMotifSearch(Dna, k, t)
Nosso algoritmo de busca de padrões gananciosos proposto, 'GreedyMotifSearch', começa por definir 'BestMotifs' igual ao primeiro k-mer de cada string em DNA. Essas cordas servirão como os padrões com melhor pontuação encontrados até agora.

Em seguida, ele percorre todos os k-mers possíveis em 'Dna[0]', experimentando cada um deles como 'Motifs[0]'. Para uma determinada escolha de k-mer em 'Dna[0]' para Motifs[0], o algoritmo constrói uma matriz de perfil Profile para esse k-mer solitário e define 'Motifs[1]' igual ao k-mer mais provável do 'Profile' em 'Dna[1]'. Em seguida, o 'GreedyMotifSearch' itera atualizando 'Profile' como a matriz de perfil formada por 'Motifs[0]' e 'Motifs[1]' e define 'Motifs[2]' como o k-mer mais provável do 'Profile' em 'Dna[2]'. Em geral, depois de encontrar k-mers 'Motifs' nas primeiras i cadeias de caracteres de 'Dna', o 'GreedyMotifSearch' constrói 'Profile(Motifs)' e define 'Motifs[i]' como o k-mer mais provável do 'Profile' em 'Dna[i]' com base nessa matriz de perfil.

Depois de selecionar um k-mer de cada cadeia de caracteres no DNA para obter uma coleção de cadeias de caracteres 'Motifs', o GreedyMotifSearch verifica se o Motifs supera a coleção de padrões com a melhor pontuação atual, BestMotifs.

Em seguida, ele retorna ao topo do loop for e move um símbolo em 'Dna[0]', iniciando todo o processo de geração de 'Motifs' novamente. Depois de gerar uma coleção de Motifs para cada k-mer inicial possível de 'Dna[0]', ele retorna as strings BestMotifs de alta pontuação.

```
GreedyMotifSearch(Dna, k, t)
    BestMotifs ← motif matrix formed by first k-mers in each string from Dna
    for each k-mer Motif in the first string from Dna
        Motif1 ← Motif
        for i = 2 to t
            form Profile from motifs Motif1, …, Motifi - 1
            Motifi ← Profile-most probable k-mer in the i-th string in Dna
        Motifs ← (Motif1, …, Motift)
        if Score(Motifs) < Score(BestMotifs)
            BestMotifs ← Motifs
    return BestMotifs
```

In [ ]:
def Consensus(Motifs):
    k = len(Motifs[0])
    count = Count(Motifs)
    consensus = ""
    for j in range(k):
        m = 0
        frequentSymbol = ""
        for symbol in "ACGT":
            if count[symbol][j] > m:
                m = count[symbol][j]
                frequentSymbol = symbol
        consensus += frequentSymbol
    return consensus

def Count(Motifs):
    count = {}
    for nt in "ACGT":
        count[nt] = [0]*len(Motifs[0])
    for motif in Motifs:
        for index, let in enumerate(motif):
            count[let][index] += 1
    return count

def Score(Motifs):
    consensus = Consensus(Motifs)
    score = 0

    for motif in Motifs:
        for j in range(len(Motifs[0])):
            if motif[j] != consensus[j]:
                score += 1
    return score

def Pr(Text, Profile):
    p = 1
    for i in range(len(Text)):
        for j in "ACGT":
            if Text[i] == j:
                p *= Profile[Text[i]][i]
    return p

def Profile(Motifs):
  # Obtém a contagem de cada nucleotídeo em cada posição
  count = Count(Motifs)
  # Número total de sequências
  num_sequences = len(Motifs)
  # Converte a contagem para perfil de frequência
  profile = {'A': [], 'C': [], 'G': [], 'T': []}
  for nucleotide in 'ACGT':
    for count_at_position in count[nucleotide]:
      # Calcula a frequência dividindo a contagem pelo número total de sequências
      frequency = count_at_position / num_sequences
      profile[nucleotide].append(frequency)
  return profile

def ProfileMostProbableKmer(text, k, profile):
    most_probable_kmer = ''
    max_probability = -1.0  # inicializar max_probability com um valor negativo, como -1.0,
    # é garantir que qualquer probabilidade calculada seja maior do que o valor inicial.
    for i in range(len(text) - k + 1):
        kmer = text[i:i+k]
        probability = 1.0  # inicia em 1 por causa das multiplicações que irão ocorrer
        # Calcular a probabilidade do k-mer com base no perfil
        for j in range(k):
            probability *= profile[kmer[j]][j]
        # Verifica se a probabilidade calculada é maior que a maior já encontrada
        if probability > max_probability:
            max_probability = probability
            most_probable_kmer = kmer

    return most_probable_kmer

def GreedyMotifSearch(Dna, k, t):
    BestMotifs = []
    for i in range(0, t):
        BestMotifs.append(Dna[i][0:k])
    n = len(Dna[0])
    for i in range(n-k+1):
        Motifs = []
        Motifs.append(Dna[0][i:i+k])
        for j in range(1, t):
            P = Profile(Motifs[0:j])
            Motifs.append(ProfileMostProbableKmer(Dna[j], k, P))  
        if Score(Motifs) < Score(BestMotifs):
                BestMotifs = Motifs
    return BestMotifs
  

# Dna, nesse exemplo. é 10 sequências do DosR_dataset 
Dna = ['GGCGTTCAGGCA',
       'AAGAATCAGTCA',
       'CAAGGAGTTCGC',
       'CACGTCAATCAC',
       'CAATAATATTCG'
       ]
k = 3
t = 5

print(GreedyMotifSearch(Dna,k,t))

### GreedyMotifSearch_2(Dna, k, t)

À primeira vista, 'GreedyMotifSearch' pode parecer um algoritmo razoável, mas não é!

O problema, no entanto, é que há tantos zeros na matriz de perfil que a probabilidade de cada 4-mer, mas "ACCT" é zero! 

Assim, a menos que "ACCT" está presente em todas as strings em DNA, há pouca chance de que 'GreedyMotifSearch' encontrará o 'motif' implantado. 
Zeros na matriz de perfil não são apenas um pequeno incômodo, mas sim um problema persistente que devemos abordar.

#### Exemplo modificado:

In [ ]:
def Consensus_2(Motifs):
    k = len(Motifs[0])
    count_2 = Count_2(Motifs)
    consensus_2 = ""
    for j in range(k):
        m = 0
        frequentSymbol = ""
        for symbol in "ACGT":
            if count_2[symbol][j] > m:
                m = count_2[symbol][j]
                frequentSymbol = symbol
        consensus_2 += frequentSymbol
    return consensus_2

def Count_2(Motifs):
    count_2 = {}
    for nt in "ACGT":
        count_2[nt] = [0]*len(Motifs[0])
    for motif in Motifs:
        for index, let in enumerate(motif):
            count_2[let][index] += 1
    return count_2

def Score(Motifs):
    consensus_2 = Consensus_2(Motifs)
    score = 0
    for motif in Motifs:
        for j in range(len(Motifs[0])):
            if motif[j] != consensus_2[j]:
                score += 1
    return score

def Pr(Text, Profile_2):
    p = 1
    for i in range(len(Text)):
        for j in "ACGT":
            if Text[i] == j:
                p *= Profile_2[Text[i]][i]
    return p

def Profile_2(Motifs):
  # Obtém a contagem de cada nucleotídeo em cada posição
  count_2 = Count_2(Motifs)
  # Número total de sequências
  num_sequences = len(Motifs)
  # Converte a contagem para perfil de frequência
  profile_2 = {'A': [], 'C': [], 'G': [], 'T': []}
  for nucleotide in 'ACGT':
    for count_2_at_position in count_2[nucleotide]:
      # Calcula a frequência dividindo a contagem pelo número total de sequências
      frequency = count_2_at_position / num_sequences
      profile_2[nucleotide].append(frequency)
  return profile_2

def ProfileMostProbableKmer_2(text, k, profile_2):
    most_probable_kmer = ''
    max_probability = -1.0  # inicializar max_probability com um valor negativo, como -1.0,
    # é garantir que qualquer probabilidade calculada seja maior do que o valor inicial.
    for i in range(len(text) - k + 1):
        kmer = text[i:i+k]
        probability = 1.0  # inicia em 1 por causa das multiplicações que irão ocorrer
        # Calcular a probabilidade do k-mer com base no perfil
        for j in range(k):
            probability *= profile_2[kmer[j]][j]
        # Verifica se a probabilidade calculada é maior que a maior já encontrada
        if probability > max_probability:
            max_probability = probability
            most_probable_kmer = kmer
    return most_probable_kmer

def GreedyMotifSearch_2(Dna, k, t):
    BestMotifs = []
    for i in range(0, t):
        BestMotifs.append(Dna[i][0:k])
    n = len(Dna[0])
    for i in range(n-k+1):
        Motifs = []
        Motifs.append(Dna[0][i:i+k])
        for j in range(1, t):
            P = Profile_2(Motifs[0:j])
            Motifs.append(ProfileMostProbableKmer_2(Dna[j], k, P))
            
        if Score(Motifs) < Score(BestMotifs):
                BestMotifs = Motifs
    return BestMotifs

# Código para colocar aspas e vírgulas na str Dna
def aspas_vir_str():
    with open("dataset_30305_5 (2).txt", "r") as file_1:
        linhas_1 = file_1.readlines()
    string_original = linhas_1[1].strip()
    palavras = string_original.split()
    return palavras


Dna = aspas_vir_str()
k = 12
t = len(Dna)
print(aspas_vir_str())
print(GreedyMotifSearch_2(Dna, k, t))